In [1]:
import pandas as pd
import json
import numpy as np
import jenks

In [65]:
momc = pd.read_csv('../scratch/momc_meta.csv').set_index('Kod').to_dict(orient='index')
ob = pd.read_csv('../scratch/obce_meta.csv').set_index('Kod').to_dict(orient='index')
okr = pd.read_csv('../scratch/okres_meta.csv').set_index('Kod').to_dict(orient='index')

In [66]:
with open('../scratch/okrsky.geojson', encoding='utf-8') as f:
    jsn = json.load(f)

In [67]:
kill = ['fid', 'gml_id', 'Kod', 'Nespravny', 'ObecKod', 'MomcKod', 'Poznamka', 'PlatiOd', 'PlatiDo', 'IdTransakce', 'GlobalniIdNavrhuZmeny']
for f in jsn['features']:
    f['properties']['Obec'] = ob[f['properties']['ObecKod']]['Nazev']
    f['properties']['Okres'] = okr[ob[f['properties']['ObecKod']]['OkresKod']]['Nazev']
    f['properties']['Momc'] = None
    f['properties']['okid'] = str(f['properties']['ObecKod']) + '_' + str(f['properties']['Cislo'])
    
    if f['properties']['MomcKod'] is not None:
        f['properties']['Momc'] = momc[f['properties']['MomcKod']]['Nazev']
        f['properties']['okid'] = str(f['properties']['MomcKod']) + '_' + str(f['properties']['Cislo'])

    for k in kill:
        if k in f['properties']:
            del f['properties'][k]

In [68]:
okrs = pd.read_csv('../scratch/okrsky_cisla.csv')

In [69]:
okrs['okid'] = okrs.CIS_OBEC.astype('str') + '_' + okrs.CIS_OKRSEK.astype('str')

In [70]:
cls = ['CIS_OBEC', 'CIS_OBVODU', 'CIS_OKRSEK', 'DATUM_CAS_ZPRAC', 'OPAKOVANE', 'PORADI_ZPRAC']
okrs.drop(cls, inplace=True, axis=1)

In [71]:
okrs = okrs.set_index('okid').to_dict(orient='index')

In [72]:
for fe in jsn['features']:
    if fe['properties']['okid'] in okrs:
        fe['properties'].update(okrs[fe['properties']['okid']])
    else:
        print(fe['properties']['okid'])

565181_1
577596_1
577596_2
549886_1
581950_1
566845_1
530662_1
592935_1
571318_1
578959_1
566837_1
549169_1
531774_1
545422_1
571075_1
532258_1
536474_1
555177_1
557722_1
557722_2
581526_1
593044_1
503941_1


In [73]:
with open('../scratch/map.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(jsn))

In [36]:
#breaks
k = 5
nbAttempt = 4

target = open('../js/breaks.js', 'w')
target.truncate()
target.write('var breaks = {\n')

for i in range(1, 10):
    print('kandidat ' + str(i))
    vals = []
    for ftr in jsn['features']:
        try:
            vals.append(ftr['properties']['HLASY_' + str(i).rjust(2, '0')] / ftr['properties']['PL_HL_CELK'])
        except:
            print('omit')
        
    vals = np.trim_zeros(vals)
    if np.mean(vals) == 0:
        continue
    else:
        brks = [0, 0, 0, 0]
        try:
            casp = jenks.jenksCaspall(vals, k, nbAttempt, initStyle='kpp')
            brks = casp.breaks
        except:
            pass
        target.write('"HLASY_' + str(i).rjust(2, '0') + '": ' + str(brks) + ',\n')
        
vals = []
print('ucast')
for ftr in jsn['features']:
    try:
        vals.append(ftr['properties']['PL_HL_CELK'] / ftr['properties']['VOL_SEZNAM'])
    except:
        print('omit')

vals = np.trim_zeros(vals)
if np.mean(vals) == 0:
    pass
else:
    brks = [0, 0, 0, 0]
    try:
        casp = jenks.jenksCaspall(vals, k, nbAttempt, initStyle='kpp')
        brks = casp.breaks
    except:
        pass
    target.write('"UCAST": ' + str(brks) + ',\n')       
target.write("};")
target.close()

kandidat 1
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
Running Jenks-Caspall natural breaks...
**Attempt number 1
Step 1 : kmeans++ initalization, GVF = 0.827233
Step 2 : kmeans complete in 51 iterations and 11357 moves, GVF = 0.899482
Step 3 : Forcing completed in 2 cycles and 4 moves, GVF = 0.899482
Running Jenks-Caspall natural breaks...
**Attempt number 2
Step 1 : kmeans++ initalization, GVF = 0.892175
Step 2 : kmeans complete in 24 iterations and 2172 moves, GVF = 0.899522
Step 3 : Forcing completed in 1 cycles and 0 moves, GVF = 0.899522
Running Jenks-Caspall natural breaks...
**Attempt number 3
Step 1 : kmeans++ initalization, GVF = 0.879736
Step 2 : kmeans complete in 42 iterations and 5052 moves, GVF = 0.899508
Step 3 : Forcing completed in 2 cycles and 6 moves, GVF = 0.899508
Running Jenks-Caspall natural breaks...
**Attempt number 4
Step 1 : kmeans++ initalization, GVF = 0.857336
Step 2 : kmeans complete in 50 iterations and 7769 moves, GVF = 0